# Predicting freq[Hz] in Lab_139_1Ch1_1_prepared

### Notebook automatically generated from your model

Model Extra trees, trained on 2021-12-25 12:40:18.

#### Generated on 2021-12-25 07:12:22.304888

prediction
This notebook will reproduce the steps for a REGRESSION on  Lab_139_1Ch1_1_prepared.
The main objective is to predict the variable freq[Hz]

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [15]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [16]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [17]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'double', u'name': u'freq[Hz]'}, {u'type': u'double', u'name': u're:Trc1_S11'}, {u'type': u'double', u'name': u'im:Trc1_S11'}, {u'type': u'double', u'name': u're:Trc2_S21'}, {u'type': u'double', u'name': u'im:Trc2_S21'}]}

ml_dataset_handle = dataiku.Dataset('Lab_139_1Ch1_1_prepared')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 8.61 ms, sys: 20.4 ms, total: 29 ms
Wall time: 78.9 ms
Base data has 601 rows and 5 columns


,freq[Hz],re:Trc1_S11,im:Trc1_S11,re:Trc2_S21,im:Trc2_S21
0,2.400000e+09,0.150847,0.112573,0.000150,-0.003453
1,2.400167e+09,0.150640,0.110470,0.000107,-0.003357
2,2.400333e+09,0.151034,0.108374,0.000086,-0.003420
3,2.400500e+09,0.151127,0.105698,0.000075,-0.003568
4,2.400667e+09,0.151428,0.103363,-0.000033,-0.003591


#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [18]:
ml_dataset = ml_dataset[[u're:Trc2_S21', u'im:Trc1_S11', u're:Trc1_S11', u'freq[Hz]', u'im:Trc2_S21']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [19]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = []
numerical_features = [u're:Trc2_S21', u'im:Trc1_S11', u're:Trc1_S11', u'im:Trc2_S21']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We renamed the target variable to a column named target

In [20]:
ml_dataset['__target__'] = ml_dataset['freq[Hz]']
del ml_dataset['freq[Hz]']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [21]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

Train data has 480 rows and 5 columns
Test data has 121 rows and 5 columns


#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [22]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'MEAN', 'feature': u're:Trc2_S21'}, {'impute_with': u'MEAN', 'feature': u'im:Trc1_S11'}, {'impute_with': u'MEAN', 'feature': u're:Trc1_S11'}, {'impute_with': u'MEAN', 'feature': u'im:Trc2_S21'}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

Imputed missing values in feature re:Trc2_S21 with value -0.00016933902256
Imputed missing values in feature im:Trc1_S11 with value -0.0960929156663
Imputed missing values in feature re:Trc1_S11 with value 0.0381466791435
Imputed missing values in feature im:Trc2_S21 with value 0.000136754167158


/Applications/DataScienceStudio.app/Contents/Resources/kit/python.packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


We can now handle the categorical features (still using the settings defined in Models):

Let's rescale numerical features

In [23]:
rescale_features = {u're:Trc2_S21': u'AVGSTD', u'im:Trc1_S11': u'AVGSTD', u're:Trc1_S11': u'AVGSTD', u'im:Trc2_S21': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

Rescaled re:Trc2_S21
Rescaled im:Trc1_S11
Rescaled re:Trc1_S11
Rescaled im:Trc2_S21


/Applications/DataScienceStudio.app/Contents/Resources/kit/python.packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [24]:
train_X = train.drop('__target__', axis=1)
test_X = test.drop('__target__', axis=1)

train_Y = np.array(train['__target__'])
test_Y = np.array(test['__target__'])

Now we can finally create our model !

In [25]:
from sklearn.ensemble import ExtraTreesRegressor
clf = ExtraTreesRegressor(
                    n_estimators=100,
                    random_state=1337,
                    max_depth=8,
                    min_samples_leaf=1,
                    verbose=2
                   )

... And train it

In [26]:
%time clf.fit(train_X, train_Y)

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.1s finished


ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=8,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
          oob_score=False, random_state=1337, verbose=2, warm_start=False)

Build up our result dataset

In [27]:
%time _predictions = clf.predict(test_X)
predictions = pd.Series(data=_predictions, index=test_X.index, name='predicted_value')

# Build scored dataset
results_test = test_X.join(predictions, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'freq[Hz]'})

CPU times: user 7.78 ms, sys: 1.43 ms, total: 9.2 ms
Wall time: 8.27 ms


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


#### Results

You can measure the model's accuracy:

In [28]:
c =  results_test[['predicted_value', 'freq[Hz]']].corr()
print ('Pearson correlation: %s' % c['predicted_value'][1])

Pearson correlation: 0.9999450901210248


That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
